Link dos datasets

Samba: https://www.kaggle.com/datasets/clovesgtx/brazilian-music-samba-lyrics

Gospel: https://raw.githubusercontent.com/damarals/letras/master/inst/csv/letras.csv

Axe: https://www.kaggle.com/datasets/jorgefjr/brazilian-songs-lyrics

Funk: https://www.kaggle.com/datasets/sleshes/funk-carioca-dataset

Bossa Nova: https://www.kaggle.com/datasets/mcarujo/bossa-nova-lyrics

Rock,Sertanejo, Samba2 = https://github.com/gabriellmd/data-frames-nlp

In [1]:
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

In [2]:
bossa = pd.read_csv('datasets/bossa_nova_songs_dataset.csv')
samba = pd.read_csv('datasets/samba_dataset.csv',sep='|')
axe = pd.read_csv('datasets/letras_mus_br_axe.csv')
funk = pd.read_csv('datasets/Dataset_funk_carioca.csv')
gospel = pd.read_csv('datasets/gospel.csv') 

In [3]:
gospel_ajustado = pd.DataFrame(gospel['letra'])  #OK 55000
gospel_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK
gospel_ajustado.drop(gospel_ajustado.tail(51498).index,inplace = True) #3000
samba_ajustado = pd.DataFrame(samba['letra'])  #OK 3000
samba_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK
samba_ajustado.drop(samba_ajustado.tail(118).index,inplace = True) #3000
axe_ajustado = pd.DataFrame(axe['letras'])  #OK1000
axe_ajustado.rename(columns={'letras': 'lyric'}, inplace = True) #OK
funk_ajustado = pd.DataFrame(funk['letra'])  #OK 1000
funk_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK
bossa_ajustado = pd.DataFrame(bossa['song_lyrics'])  #OK 6000
bossa_ajustado.rename(columns={'song_lyrics': 'lyric'}, inplace = True) #OK
bossa_ajustado.drop(bossa_ajustado.tail(3106).index,inplace = True) #3000

In [4]:
bossa_ajustado['gen'] = 'Bossa Nova'
funk_ajustado['gen'] = 'Funk'
gospel_ajustado['gen'] = 'Gospel'
axe_ajustado['gen'] = 'Axe'
samba_ajustado['gen'] = 'Samba'

In [5]:
datasets = [bossa_ajustado,funk_ajustado,bossa_ajustado,axe_ajustado,samba_ajustado]

In [6]:
lyrics = pd.concat(datasets)
lyrics.reset_index(drop=True,inplace=True)
lyrics

,lyric,gen
0,"De tudo, ao meu amor serei atento antes E co...",Bossa Nova
1,Era uma casa Muito engraçada Não tinha teto ...,Bossa Nova
2,E por falar em saudade Onde anda você Onde a...,Bossa Nova
3,Filhos... Filhos? Melhor não tê-los! Mas se ...,Bossa Nova
4,É melhor ser alegre que ser triste Alegria é...,Bossa Nova
...,...,...
10995,( Não tenho lar Não tenho amor Pra que vou me ...,Samba
10996,o Senhor é a minha forçao meu escudo...minha ...,Samba
10997,Sorte é sorte o malandro também tem seu dia d...,Samba
10998,"A gente briga sem querer Chora ""prá"" voltar F...",Samba


## Novos Parâmetros

In [7]:
import operator
def rima(versos):
    rim = ['', '', '', '']
    padroes = {}
    for i in range(0, len(versos), 4):
        atual = [versos[i][-1], versos[i+1][-1], versos[i+2][-1], versos[i+3][-1]]
        tam_a = len(atual[0])
        tam_b = len(atual[1])
        tam_c = len(atual[2])
        tam_d = len(atual[3])
        a = 'A' if tam_a >= 2 else '?'
        b = 'B' if tam_b >= 2 else '?'
        c = 'C' if tam_c >= 2 else '?'
        d = 'D' if tam_d >= 2 else '?'
        if a == '?':
            rim[0] = '?'
        if b == '?':
            rim[1] = '?'
        if c == '?':
            rim[2] = '?'
        if d == '?':
            rim[3] = '?'
            
        if rim[0] == '':
            rim[0] = a
            if rim[1] == '':
                if atual[0][tam_a-2:] == atual[1][tam_b-2:]:
                    b = a
                    rim[1] = b
            if rim[2] == '':
                if atual[0][tam_a-2:] == atual[2][tam_c-2:]:
                    c = a
                    rim[2] = c
            if rim[3] == '':
                if atual[0][tam_a-2:] == atual[3][tam_d-2:]:
                    d = a
                    rim[3] = d
        if rim[1] == '':
            rim[1] = b
            if rim[2] == '':
                if atual[1][tam_b-2:] == atual[2][tam_c-2:]:
                    c = b
                    rim[2] = c
            if rim[3] == '':
                if atual[1][tam_b-2:] == atual[3][tam_d-2:]:
                    d = b
                    rim[3] = d
        if rim[2] == '':
            rim[2] = c
            if rim[3] == '':
                if atual[2][tam_c-2:] == atual[3][tam_d-2:]:
                    d = c
                    rim[3] = d
        if rim[3] == '':
            rim[3] = d
        rim_atual = rim[0] + rim[1] + rim[2] + rim[3]
        if rim_atual in padroes.keys():
            padroes[rim_atual] += 1
        else:
            padroes[rim_atual] = 1
    
    rim_final = max(padroes.items(), key=operator.itemgetter(1))[0]
    return rim_final

In [8]:
rimas = []
qtd_words = []
for i in lyrics.index:
    words = str(lyrics['lyric'][i]).split()
    versos = []
    rim = '????'
    qtd = len(words)
    for i in range(len(words)):
        word = words[i]
        if word.istitle() or not versos:
              versos.append([word])
        else:
              versos[-1].append(word)
    tam = len(versos)
    if tam > 4:
        versos = versos[:tam - (tam % 4)]
        rim = rima(versos)
    rimas.append(rim)
    qtd_words.append(qtd)
lyrics['rima'] = rimas
lyrics['qtd_words'] = qtd_words


In [9]:
lyrics

,lyric,gen,rima,qtd_words
0,"De tudo, ao meu amor serei atento antes E co...",Bossa Nova,ABBB,103
1,Era uma casa Muito engraçada Não tinha teto ...,Bossa Nova,ABCB,54
2,E por falar em saudade Onde anda você Onde a...,Bossa Nova,ABCD,107
3,Filhos... Filhos? Melhor não tê-los! Mas se ...,Bossa Nova,ABCD,138
4,É melhor ser alegre que ser triste Alegria é...,Bossa Nova,?ACD,572
...,...,...,...,...
10995,( Não tenho lar Não tenho amor Pra que vou me ...,Samba,????,188
10996,o Senhor é a minha forçao meu escudo...minha ...,Samba,????,47
10997,Sorte é sorte o malandro também tem seu dia d...,Samba,??CD,427
10998,"A gente briga sem querer Chora ""prá"" voltar F...",Samba,ABAB,117


In [10]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
def processamento_texto(txt):
    if isinstance(txt, str):
        # remover quebras de linha
        txt = txt.replace('\n',' ')
        # remover símbolos de pontuação, resultando em um array de caracteres
        txt = [char for char in txt if char not in string.punctuation]
        # depois, juntar os caracteres em palavras novamente e separá-los em uma lista de tokens
        txt = ''.join(txt).split()
        # por fim, remover as stopwords da lista
        stop_words = set(stopwords.words('portuguese'))
        txt = [word for word in txt if word.lower() not in stop_words]
    
    return txt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luizm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
alg ={'SVM':{
    'fit_time':np.array([]),
    'score_time':np.array([]),
    'test_accuracy':np.array([]),
    'test_f1_score':np.array([]),
},
     'RF':{
    'fit_time':np.array([]),
    'score_time':np.array([]),
    'test_accuracy':np.array([]),
    'test_f1_score':np.array([]),
},
     'NB':{
    'fit_time':np.array([]),
    'score_time':np.array([]),
    'test_accuracy':np.array([]),
    'test_f1_score':np.array([]),
} }

In [ ]:
for count in range(5):
    #aleatorio
    lyric_train,lyric_test,gen_train,gen_test = train_test_split(lyrics[['lyric', 'rima', 'qtd_words']],lyrics['gen'],test_size=0.3)
    #dados SVC
    pipeline_svc = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto,encoding ='unicode')),
    ('tfidf',TfidfTransformer()),
    ('svc',SVC())
    ])
    #dados NB
    pipeline_nb = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto,encoding ='unicode')),
    ('tfidf',TfidfTransformer()),
    ('nb',MultinomialNB())
    ])
    #dadps RF
    pipeline_rf = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto,encoding ='unicode')),
    ('tfidf',TfidfTransformer()),
    ('rf',RandomForestClassifier())
    ])
    
    #Etapa de Fit
    pipeline_svc.fit(lyric_train.values.astype('U'),gen_train.values.astype('U'))
    pipeline_nb.fit(lyric_train.values.astype('U'),gen_train.values.astype('U'))
    pipeline_rf.fit(lyric_train.values.astype('U'),gen_train.values.astype('U'))
    
    #Etapa 10-fold cross validation
    scoring = {'accuracy' : 'accuracy', 'f1_score' : 'f1_micro'}
    scores_svc = cross_validate(pipeline_svc,lyrics[['lyric', 'rima', 'qtd_words']].values.astype('U'),lyrics['gen'].values.astype('U'),cv=10,scoring=scoring)
    scores_nb = cross_validate(pipeline_nb,lyrics[['lyric', 'rima', 'qtd_words']].values.astype('U'),lyrics['gen'].values.astype('U'),cv=10,scoring=scoring)
    scores_rf = cross_validate(pipeline_rf,lyrics[['lyric', 'rima', 'qtd_words']].values.astype('U'),lyrics['gen'].values.astype('U'),cv=10,scoring=scoring)
    
    #Aninhamento de médias
    #SVM
    alg['SVM']['fit_time'] = np.append(alg['SVM']['fit_time'], scores_svm['fit_time'].mean())
    alg['SVM']['score_time'] = np.append(alg['SVM']['score_time'], scores_svm['score_time'].mean())
    alg['SVM']['test_accuracy'] = np.append(alg['SVM']['test_accuracy'], scores_svm['test_accuracy'].mean())
    alg['SVM']['test_f1_score'] = np.append(alg['SVM']['test_f1_score'], scores_svm['test_f1_score'].mean())
    #NB
    alg['NB']['fit_time'] = np.append(alg['NB']['fit_time'], scores_nb['fit_time'].mean())
    alg['NB']['score_time'] = np.append(alg['NB']['score_time'], scores_nb['score_time'].mean())
    alg['NB']['test_accuracy'] = np.append(alg['NB']['test_accuracy'], scores_nb['test_accuracy'].mean())
    alg['NB']['test_f1_score'] = np.append(alg['NB']['test_f1_score'], scores_nb['test_f1_score'].mean())
    #RF
    alg['RF']['fit_time'] = np.append(alg['RF']['fit_time'], scores_rf['fit_time'].mean())
    alg['RF']['score_time'] = np.append(alg['RF']['score_time'], scores_rf['score_time'].mean())
    alg['RF']['test_accuracy'] = np.append(alg['RF']['test_accuracy'], scores_rf['test_accuracy'].mean())
    alg['RF']['test_f1_score'] = np.append(alg['RF']['test_f1_score'], scores_rf['test_f1_score'].mean())
    